In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part'):
    shutil.rmtree('./file/2.part')

os.makedirs('./file/2.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.url/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(input_.loc[a, 'Url'],
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(resp.text, 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                title = html.xpath('//head/title/text()')[0].strip()

                # = = = = = = = = = = = = = = =

                type_ = html.xpath('//span[@itemprop="name"]')[-1].xpath('./text()')[0].strip()

                # = = = = = = = = = = = = = = =

                tag = html.xpath('//p[@class="product-title"]/text()')[0].strip() if html.xpath('//p[@class="product-title"]/text()') else ''

                # = = = = = = = = = = = = = = =

                dict_description, list_div = {}, html.xpath('//div[@id="product-description-tab"]/div/ul/li')
                for div in list_div:
                    list_text = [text.strip().removesuffix(':').strip() for text in div.xpath('./span/text()')]
                    if len(list_text) == 0:
                        continue
                    elif len(list_text) == 1:
                        raise
                    elif len(list_text) == 2:
                        dict_description[list_text[0]] = list_text[1]
                    else:
                        dict_description[list_text[0]] = {}
                        for i, text in zip(range(len(list_text[1:])), list_text[1:]):
                            dict_description[list_text[0]][str(i)] = text
                
                # = = = = = = = = = = = = = = =

                list_specification = html.xpath('//div[@id="product-description-tab"]/specifications/specification')
                for i in range(len(list_specification)):
                    dict_description[list_specification[i].xpath('./text()')[0].strip()] = html.xpath('//div[@id="product-description-tab"]/specifications/value')[i].xpath('./text()')[0].strip()

                # = = = = = = = = = = = = = = =

                list_p = html.xpath('//div[@id="product-description-tab"]/p')
                for p in list_p:
                    if p.xpath('./strong/text()'):
                        key = p.xpath('./strong/text()')[0].strip().removesuffix(':').strip()
                        if key == 'Item Condition':
                            dict_description[p.xpath('./strong/text()')[0].strip().removesuffix(':').strip()] = p.xpath('./span/text()')[0].strip()
                        elif key == 'WARNING':
                            dict_description[p.xpath('./strong/text()')[0].strip().removesuffix(':').strip()] = p.xpath('./text()')[0].strip()

                # = = = = = = = = = = = = = = =

                part_code = int(resp.text.split('window.productId =')[1].split(';')[0].strip())
    
                # = = = = = = = = = = = = = = =

                json_ = json.loads(resp.text.split('<!-- Google structured data  Markup -->')[1].split('<script type="application/ld+json">')[1].split('</script>')[0])
    
                # = = = = = = = = = = = = = = =

                dict_src, list_src = {}, json_['image']
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = f'''https:{src.strip()}'''

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Type': type_,
                                         'Part Number': json_['mpn'].strip(),
                                         'Brand': json_['brand']['name'].strip(),
                                         'Tag': tag,
                                         'Title': title,
                                         'SKU': json_['sku'].strip(),
                                         'Price': f'''{json_['offers']['priceCurrency'].strip()} {json_['offers']['price']}''',
                                         'Rating': json_['aggregateRating']['ratingValue'].strip() if json_['aggregateRating'] else '',
                                         'Rating Count': int(json_['aggregateRating']['ratingCount'].strip()) if json_['aggregateRating'] else 0,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Json_Src': json.dumps(dict_src),
                                         'Json_Description': json.dumps(dict_description),
                                         'Part Code': part_code}])
                
                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part/{file}',
              f'''./file/2.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：4

[状态：ok，尝试次数：1] - 50.00% - crawler_1-part_error > 7400.https://www.1aauto.com/brake-caliper-piston-rewind-cube/i/1axaa00441
[剩余数量：0] - [当前时间：08:47:22]

[状态：ok，尝试次数：3] - 50.00% - crawler_1-part_error > 7397.https://www.1aauto.com/disc-brake-service-kit-50-state-formula/i/1avck00001
[剩余数量：0] - [当前时间：08:47:22]

[状态：ok，尝试次数：2] - 50.00% - crawler_1-part_error > 7398.https://www.1aauto.com/brake-caliper-service-kit/i/1avck00018
[剩余数量：0] - [当前时间：08:47:23]

[状态：ok，尝试次数：10] - 50.00% - crawler_1-part_error > 7399.https://www.1aauto.com/2-piece-brake-caliper-parts-hanger-hook-set/i/1axaa00439
[剩余数量：0] - [当前时间：08:47:25]

输出ing...

总数量：8

[状态：ok，尝试次数：7] - 100.00% - crawler_2-part_error > 727.https://www.1aauto.com/1982-92-chevrolet-camaro-pontiac-firebird-4-piece-t-top-weatherstrip-seal-kit-fairchild-automotive-1awsk00167/i/1awsk00167
[剩余数量：3] - [当前时间：08:47:28]

[状态：ok，尝试次数：12] - 100.00% - crawler_2-part_error > 657.https://www.1aauto.com/1978-81-chevrolet-pontiac-roofrail-weatherstrip-seal-2

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3780.36it/s]

Done ~
